In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random
import numpy as np
from astropy.io import ascii
from astropy.table import Table
import csv


#open the file
datapath='/home/lee/research/references/'
filename='flux_table_70um.txt'
data=np.loadtxt(datapath+filename)

#number is coloumn 0, flux is column 25
number=data[:,0]
flux=data[:,25]

In [2]:
#remove zeros in the flux
flux_new=[]
for i in range(len(flux)):
    if flux[i]!=0:
        flux_new.append(flux[i])

run=50
t=300
cadence=26

In [3]:
run=50
t=300
cadence=26

#run it 50 times
#start with function
def bursts1(full_time,size,probability,L_matrix,burst_matrix,change_matrix):
    cluster_flux=[]
    star=[]
    cluster_size=size
    for i in range(cluster_size):
        cluster_flux.append(random.choice(flux_new))
        star.append(i)
    cluster_mag=[]
    for i in range(len(cluster_flux)):
        mag=-2.5*np.log10(cluster_flux[i])
        cluster_mag.append(mag)
    #make multi dimensional array for burst prob 
    #let 1 be unburst and NaN be burst
    t=full_time #yrs
    cadence=26
    steps=t*cadence
    burst_prob=probability/cadence
    burst=burst_matrix
    L=L_matrix
    C=change_matrix
    for i in range(steps):
        for j in range(cluster_size):
            if burst[j,i]==1:
                prob=random.random()
                if prob>burst_prob:
                    burst[j,i]=burst[j,i]
                    L[j,i]=cluster_mag[j]
                    C[j,i]=0
                elif prob<=burst_prob:
                    burst[j,i:i+2]=np.nan
                    L[j,i:i+2]=cluster_mag[j]-1.25
                    C[j,i:i+2]=3
    #determine total number of bursts 
    total=0
    burst_total=[]
    for i in range(steps):
        for j in range(cluster_size):
            if np.isnan(burst[j,i])==True and burst[j,i-1]==1:
                    total+=1
        burst_total.append(total)
    #find the fraction at a time
    burst_num=[]
    fraction_val=0
    for i in range(steps):
        for j in range(cluster_size):
            if np.isnan(burst[j,i])==True:
                fraction_val+=1
        burst_num.append(fraction_val)
        fraction_val=0
        
    return burst_total,burst_num,burst,L,C

In [4]:
t_list_years=[200,220,220.5,221,221.5,222,222.5,223,223.5,224,224.5,225]
t_list_cadence=[int(i*2) for i in t_list_years]

In [5]:
print(365/14)

26.071428571428573


## 50 stars

In [6]:
cluster_size50=50
steps=t*cadence
L50=np.zeros((cluster_size50,steps))
C50=np.zeros((cluster_size50,steps))
duration=14/365
prob50=1/(3*duration)

#1 bin
burst_total50=[]
burst_num50=[]
final_burst50=[]
final_L50=[]
final_C50=[]

for i in range(run):
    print(i)
    burst50=np.ones((cluster_size50,steps))
    L50=np.zeros((cluster_size50,steps))
    C50=np.zeros((cluster_size50,steps))
    values50=bursts1(t,50,prob50,L50,burst50,C50)
    burst_total50.append(values50[0])
    burst_num50.append(values50[1])
    final_burst50.append(values50[2])
    final_L50.append(values50[3])
    final_C50.append(values50[4])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [7]:
#write output files
values50_table=Table()
values50_table['burst_total']=burst_total50
values50_table['burst_number']=burst_num50 
ascii.write(values50_table,'/home/lee/data/cluster_size_change/3x_burst/values50_3x',\
            format='ecsv',overwrite=True)

np.save('/home/lee/data/cluster_size_change/3x_burst/burst50_3x.npy',final_burst50)
np.save('/home/lee/data/cluster_size_change/3x_burst/L50_3x.npy',final_L50)
np.save('/home/lee/data/cluster_size_change/3x_burst/C50_3x.npy',final_C50)

In [8]:
print(final_L50)

[array([[-0.09831766, -0.09831766, -0.09831766, ...,  1.15168234,
         1.15168234,  1.15168234],
       [-2.76802492, -2.76802492, -2.76802492, ..., -2.76802492,
        -2.76802492, -4.01802492],
       [ 3.22852289,  3.22852289,  1.97852289, ...,  3.22852289,
         3.22852289,  1.97852289],
       ...,
       [-1.13406018, -2.38406018, -2.38406018, ..., -2.38406018,
        -2.38406018, -2.38406018],
       [-3.43089896, -4.68089896, -4.68089896, ..., -4.68089896,
        -4.68089896, -4.68089896],
       [-0.0965504 , -0.0965504 , -0.0965504 , ..., -0.0965504 ,
        -1.3465504 , -1.3465504 ]]), array([[-1.34643433, -1.34643433, -1.34643433, ..., -1.34643433,
        -2.59643433, -2.59643433],
       [ 0.42256651,  0.42256651, -0.82743349, ...,  0.42256651,
        -0.82743349, -0.82743349],
       [-1.16531468, -1.16531468, -1.16531468, ..., -1.16531468,
        -1.16531468, -1.16531468],
       ...,
       [ 3.03134881,  1.78134881,  1.78134881, ...,  1.78134881,
        

In [9]:
cadence_num_bursts50=[]
for i in range(run):
    for j in t_list_cadence:
        cadence_num_bursts50.append(burst_num50[i][j])

## 100 stars

In [10]:
cluster_size100=100
steps=t*cadence
L100=np.zeros((cluster_size100,steps))
C100=np.zeros((cluster_size100,steps))
duration=14/365
prob100=1/(3*duration)

#1 bin
burst_total100=[]
burst_num100=[]
final_burst100=[]
final_L100=[]
final_C100=[]

for i in range(run):
    print(i)
    burst100=np.ones((cluster_size100,steps))
    L100=np.zeros((cluster_size100,steps))
    C100=np.zeros((cluster_size100,steps))
    values100=bursts1(t,100,prob100,L100,burst100,C100)
    burst_total100.append(values100[0])
    burst_num100.append(values100[1])
    final_burst100.append(values100[2])
    final_L100.append(values100[3])
    final_C100.append(values100[4])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [11]:
#write output files
values100_table=Table()
values100_table['burst_total']=burst_total100
values100_table['burst_number']=burst_num100 
ascii.write(values100_table,'/home/lee/data/cluster_size_change/3x_burst/values100_3x',\
            format='ecsv',overwrite=True)

np.save('/home/lee/data/cluster_size_change/3x_burst/burst100_3x.npy',final_burst100)
np.save('/home/lee/data/cluster_size_change/3x_burst/L100_3x.npy',final_L100)
np.save('/home/lee/data/cluster_size_change/3x_burst/C100_3x.npy',final_C100)

In [12]:
cadence_num_bursts100=[]
for i in range(run):
    for j in t_list_cadence:
        cadence_num_bursts100.append(burst_num100[i][j])

## 500 stars

In [13]:
cluster_size500=500
steps=t*cadence
L500=np.zeros((cluster_size500,steps))
C500=np.zeros((cluster_size500,steps))
duration=14/365
prob500=1/(3*duration)

#1 bin
burst_total500=[]
burst_num500=[]
final_burst500=[]
final_L500=[]
final_C500=[]

for i in range(run):
    print(i)
    burst500=np.ones((cluster_size500,steps))
    L500=np.zeros((cluster_size500,steps))
    C500=np.zeros((cluster_size500,steps))
    values500=bursts1(t,500,prob500,L500,burst500,C500)
    burst_total500.append(values50[0])
    burst_num500.append(values500[1])
    final_burst500.append(values500[2])
    final_L500.append(values500[3])
    final_C500.append(values500[4])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [14]:
#write output files
values500_table=Table()
values500_table['burst_total']=burst_total500
values500_table['burst_number']=burst_num500 
ascii.write(values500_table,'/home/lee/data/cluster_size_change/3x_burst/values500_3x',\
            format='ecsv',overwrite=True)

np.save('/home/lee/data/cluster_size_change/3x_burst/burst500_3x.npy',final_burst500)
np.save('/home/lee/data/cluster_size_change/3x_burst/L500_3x.npy',final_L500)
np.save('/home/lee/data/cluster_size_change/3x_burst/C500_3x.npy',final_C500)

In [15]:
cadence_num_bursts500=[]
for i in range(run):
    for j in t_list_cadence:
        cadence_num_bursts500.append(burst_num500[i][j])

## 1000 stars

In [16]:
cluster_size1000=1000
steps=t*cadence
L1000=np.zeros((cluster_size1000,steps))
C1000=np.zeros((cluster_size1000,steps))
duration=14/365
prob1000=1/(3*duration)

#1 bin
burst_total1000=[]
burst_num1000=[]
final_burst1000=[]
final_L1000=[]
final_C1000=[]

for i in range(run):
    print(i)
    burst1000=np.ones((cluster_size1000,steps))
    L1000=np.zeros((cluster_size1000,steps))
    C1000=np.zeros((cluster_size1000,steps))
    values1000=bursts1(t,1000,prob1000,L1000,burst1000,C1000)
    burst_total1000.append(values1000[0])
    burst_num1000.append(values1000[1])
    final_burst1000.append(values1000[2])
    final_L1000.append(values1000[3])
    final_C1000.append(values1000[4])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [17]:
#write output files
values1000_table=Table()
values1000_table['burst_total']=burst_total1000
values1000_table['burst_number']=burst_num1000
ascii.write(values1000_table,'/home/lee/data/cluster_size_change/3x_burst/values1000_3x',\
            format='ecsv',overwrite=True)

np.save('/home/lee/data/cluster_size_change/3x_burst/burst1000_3x.npy',final_burst1000)
np.save('/home/lee/data/cluster_size_change/3x_burst/L1000_3x.npy',final_L1000)
np.save('/home/lee/data/cluster_size_change/3x_burst/C1000_3x.npy',final_C1000)

In [18]:
cadence_num_bursts1000=[]
for i in range(run):
    for j in t_list_cadence:
        cadence_num_bursts1000.append(burst_num1000[i][j])

## 2000 stars

In [19]:
cluster_size2000=2000
steps=t*cadence
L2000=np.zeros((cluster_size2000,steps))
C2000=np.zeros((cluster_size2000,steps))
duration=14/365
prob2000=1/(3*duration)

#1 bin
burst_total2000=[]
burst_num2000=[]
final_burst2000=[]
final_L2000=[]
final_C2000=[]

for i in range(run):
    print(i)
    burst2000=np.ones((cluster_size2000,steps))
    L2000=np.zeros((cluster_size2000,steps))
    C2000=np.zeros((cluster_size2000,steps))
    values2000=bursts1(t,2000,prob2000,L2000,burst2000,C2000)
    burst_total2000.append(values2000[0])
    burst_num2000.append(values2000[1])
    final_burst2000.append(values2000[2])
    final_L2000.append(values2000[3])
    final_C2000.append(values2000[4])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [20]:
#write output files
values2000_table=Table()
values2000_table['burst_total']=burst_total2000
values2000_table['burst_number']=burst_num2000
ascii.write(values2000_table,'/home/lee/data/cluster_size_change/3x_burst/values2000_3x',\
            format='ecsv',overwrite=True)

np.save('/home/lee/data/cluster_size_change/3x_burst/burst2000_3x.npy',final_burst2000)
np.save('/home/lee/data/cluster_size_change/3x_burst/L2000_3x.npy',final_L2000)
np.save('/home/lee/data/cluster_size_change/3x_burst/C2000_3x.npy',final_C2000)

In [21]:
cadence_num_bursts2000=[]
for i in range(run):
    for j in t_list_cadence:
        cadence_num_bursts2000.append(burst_num2000[i][j])

## 5000 stars

In [5]:
cluster_size5000=5000
steps=t*cadence
L5000=np.zeros((cluster_size5000,steps))
C5000=np.zeros((cluster_size5000,steps))
duration=14/365
prob5000=1/(3*duration)

#1 bin
burst_total5000=[]
burst_num5000=[]
final_burst5000=[]
final_L5000=[]
final_C5000=[]

for i in range(run):
    print(i)
    burst5000=np.ones((cluster_size5000,steps))
    L5000=np.zeros((cluster_size5000,steps))
    C5000=np.zeros((cluster_size5000,steps))
    values5000=bursts1(t,5000,prob5000,L5000,burst5000,C5000)
    burst_total5000.append(values5000[0])
    burst_num5000.append(values5000[1])
    final_burst5000.append(values5000[2])
    final_L5000.append(values5000[3])
    final_C5000.append(values5000[4])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [6]:
#write output files
values5000_table=Table()
values5000_table['burst_total']=burst_total5000
values5000_table['burst_number']=burst_num5000
ascii.write(values5000_table,'/home/lee/data/cluster_size_change/3x_burst/values5000_3x',\
            format='ecsv',overwrite=True)

np.save('/home/lee/data/cluster_size_change/3x_burst/burst5000_3x.npy',final_burst5000)
np.save('/home/lee/data/cluster_size_change/3x_burst/L5000_3x.npy',final_L5000)
np.save('/home/lee/data/cluster_size_change/3x_burst/C5000_3x.npy',final_C5000)

In [7]:
cadence_num_bursts5000=[]
for i in range(run):
    for j in t_list_cadence:
        cadence_num_bursts5000.append(burst_num5000[i][j])

In [8]:
#calculate the mean of each time step
t_list_years=[200,220,220.5,221,221.5,222,222.5,223,223.5,224,224.5,225]
t_list_cadence=[int(i*2) for i in t_list_years]

t_list_years=[200,220,220.5,221,221.5,222,222.5,223,223.5,224,224.5,225]
t_list_cadence=[int(i*2) for i in t_list_years]